In [1]:
import torch
from torchvision.datasets import ImageFolder
import torchvision.models as models
import torch.nn as nn
import torch.optim as optim
from tqdm import tqdm
from torchvision import datasets, transforms

In [2]:
from google.colab import drive
drive.mount("/content/drive")


Mounted at /content/drive


In [3]:
cd '/content/drive/MyDrive/colab'

/content/drive/MyDrive/colab


In [4]:
transform=transforms.Compose(
     [ transforms.PILToTensor(),
       transforms.ConvertImageDtype(torch.float),
      transforms.Normalize(mean=[0.485, 0.456, 0.406],
                         std=[0.229, 0.224, 0.225])])
train_dataset = datasets.ImageFolder('train', transform=transform)
test_dataset = datasets.ImageFolder('test', transform=transform)
val_dataset=datasets.ImageFolder('val', transform=transform)

train_data_loader = torch.utils.data.DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_data_loader=torch.utils.data.DataLoader(val_dataset, batch_size=32, shuffle=True, num_workers=2)
test_data_loader = torch.utils.data.DataLoader(test_dataset, batch_size=32, shuffle=True, num_workers=2)

In [5]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [6]:
model=models.googlenet(models.GoogLeNet_Weights)

/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:136: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and will be removed in 0.15. Please use keyword parameter(s) instead.
  f"Using {sequence_to_str(tuple(keyword_only_kwargs.keys()), separate_last='and ')} as positional "
/usr/local/lib/python3.7/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and will be removed in 0.15. The current behavior is equivalent to passing `weights=GoogLeNet_Weights.IMAGENET1K_V1`. You can also use `weights=GoogLeNet_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/googlenet-1378be20.pth" to /root/.cache/torch/hub/checkpoints/googlenet-1378be20.pth


  0%|          | 0.00/49.7M [00:00<?, ?B/s]

In [12]:
out_dim=4
model.fc=nn.Sequential(
            nn.Linear(1024,100),
            nn.ReLU(inplace=True),
            nn.Linear(100,100),
            nn.ReLU(inplace=True),
            nn.Linear(100,out_dim),
            nn.Softmax(dim=1))

In [8]:
def test(data_loader, model):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for x, y in data_loader:
            x = x.to(device)
            y = y.to(device)
            output = model(x)
            _, predictions = torch.max(output, 1)
            correct += (predictions == y).sum().item()
            
    print("Accuracy: ", correct, " / ",
    len(data_loader.dataset), "  ", int(correct / len(data_loader.dataset) * 100), "%")

In [9]:

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)

def train(num_epoch,model):
    model.to(device)
    for epoch in range(num_epoch):
        losses = []
        loop = tqdm(enumerate(train_data_loader), total=len(train_data_loader)) 
        count=0
        for batch_idx, (data, targets) in loop:
            model.train()
            data=data.to(device)
            targets=targets.to(device)
            scores=model(data)
            loss = criterion(scores, targets)
            optimizer.zero_grad()
            losses.append(loss)
            loss.backward()
            optimizer.step()
            _, preds = torch.max(scores, 1)
            
            loop.set_description(f"Epoch {epoch+1}/{num_epoch} process: {int((batch_idx / len(train_data_loader)) * 100)}")
            loop.set_postfix(loss=loss.data.item())
            count+=1
            if count%10==0:
                print("val ki accuarcy.")
                test(val_data_loader, model)
                # print("\ntest ki accuracy.")
                # test(test_data_loader) 
                
    file_name=str(input("file_name:-"))
    torch.save(model, f'accuracy_{file_name}.pth')

In [13]:
model.to('cuda')
train(5, model)

Epoch 1/5 process: 7:   7%|▋         | 9/126 [01:12<17:30,  8.98s/it, loss=1.37]

val ki accuarcy.


Epoch 1/5 process: 7:   9%|▊         | 11/126 [03:44<1:10:48, 36.94s/it, loss=1.37]

Accuracy:  197  /  472    41 %


Epoch 1/5 process: 15:  15%|█▌        | 19/126 [04:26<15:09,  8.50s/it, loss=1.36]

...

val ki accuarcy.


Epoch 2/5 process: 71:  72%|███████▏  | 91/126 [00:31<00:18,  1.88it/s, loss=1.11]

Accuracy:  463  /  472    98 %


Epoch 2/5 process: 76:  78%|███████▊  | 98/126 [00:32<00:09,  3.04it/s, loss=1.1]


KeyboardInterrupt: ignored

In [15]:
test(test_data_loader, model)

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3529549440>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1493, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f3529549440>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1510, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1493, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/pytho

Accuracy:  501  /  506    99 %


In [16]:
torch.save(model, f'perfect_googlenet.pth')